# Full Universe Beta Calculation
This notebook will study the parameters of 6 different weekly models:

| Dep var: | With Earnings | Without Earnings |
|:--------:|:-------------:|:----------------:|
| ret_cc   | Model 1       | Model 2          |
| ret_oc   | Model 3       | Model 4          |
| ret_co   | Model 5       | Model 6          |

The dependent variable will indicate the beginning and end of the testing period. ret_cc measures from close to close. ret_oc measures from open to close (i.e. while the market is open), etc.

In even numbered models, the weeks of earnings releases and the subsequent weeks are removed from the data set.

Each model will be run over the following time periods:

1. 2011
1. 2012
1. 2013
1. 2014
1. 2015
1. 2011 - 2015

## Key Questions:

1. Does the beta-lag effect exist in all models?
1. Are the companies which display beta-lags consistent from year to year?
1. Is the effect robust to earnings?
1. Does it occur in both large and small companies?

# Setup
## Import Packages

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from pandas_datareader import data
import statsmodels.formula.api as sm
import time
import wmcm
import finsymbols

## Define Universe
The tickers associated with the NYSE and the NASDAQ were pulled using the finsymbols package.

In [2]:
nyse = pd.DataFrame(finsymbols.get_nyse_symbols())
nyse['exchange'] = 'nyse'
nasdaq = pd.DataFrame(finsymbols.get_nasdaq_symbols())
nasdaq['exchange'] = 'nasdaq'
universe = pd.concat([nyse, nasdaq])
universe['symbol'] = universe['symbol'].replace('.', '-')
universe.head()

,company,industry,sector,symbol,exchange
0,3D Systems Corporation,Computer Software: Prepackaged Software,Technology,DDD,nyse
1,3M Company,Medical/Dental Instruments,Health Care,MMM,nyse
2,500.com Limited,Services-Misc. Amusement & Recreation,Consumer Services,WBAI,nyse
3,58.com Inc.,Computer Software: Programming,Technology,WUBA,nyse
4,A.H. Belo Corporation,Newspapers/Magazines,Consumer Services,AHC,nyse


## Pull Universe

In [3]:
# stocks = wmcm.Universe(universe['symbol'], 'SPY', interval='w', verbose=False)

# stocks.save('pickle/full_price_history.p')

stocks = wmcm.Universe.load('pickle/full_price_history.p')

In [4]:
stocks['GOOG']

Stock : GOOG
        Starting Date : 2011-01-01 00:00:00
        Ending Date : 2015-12-31 00:00:00
        Frequency : w

In [5]:
stocks['YCB']

Stock : YCB
        Starting Date : 2011-01-01 00:00:00
        Ending Date : 2015-12-31 00:00:00
        Frequency : w

## Prepare Analysis Dataframes

In [6]:
for key in list(stocks.keys()): #list(stock.keys()) allows you to delete while iterating
    if key=='market':
        continue
    if len(stocks[key].earnings_df.index)==0: # Have to remove stocks without earnings.
        del stocks[key]
    else:
        stocks[key].analysis_df = stocks[key].adj_returns
        stocks[key].analysis_df['lagged_ret_cc'] = stocks[key].analysis_df['ret_cc'].shift(1)
        stocks[key].analysis_df = stocks[key].analysis_df.loc[(stocks[key].analysis_df['earnings_period'].shift(1)==False)]

# Fit Models
## Model 1

In [7]:
stocks.factor_model_all('ret_cc ~ ret_cc_market + lagged_ret_cc', filter_earnings=True)
stocks.model_1 = stocks.results
stocks.model_1['GOOG']

AttributeError: 'Market' object has no attribute 'analysis_df'